# CSC582: Relational to Key-Value Database Migration
## E-Commerce Database → Redis Cluster
**Mapping Format:** `TableName:TupleID:Attribute` → Value

**Cluster:** 4 Masters + 4 Replicas

## Step 1: Install Redis

In [ ]:
!apt-get update -qq
!apt-get install -y redis-server redis-tools > /dev/null
!pip install redis -q
print('✅ Redis installed!')

## Step 2: Start 8 Redis Nodes (4 Masters + 4 Replicas)

In [ ]:
import subprocess, time, os
for port in range(7000, 7008):
    os.makedirs(f'/tmp/redis-{port}', exist_ok=True)
    subprocess.run(['redis-server', '--port', str(port), '--daemonize', 'yes', '--dir', f'/tmp/redis-{port}'])
    print(f'Started node on port {port}')
time.sleep(2)
print('\n✅ All 8 nodes started!')

In [ ]:
# Configure Replication
for replica, master in [(7004,7000), (7005,7001), (7006,7002), (7007,7003)]:
    subprocess.run(['redis-cli', '-p', str(replica), 'REPLICAOF', 'localhost', str(master)])
    print(f'Replica {replica} → Master {master}')
time.sleep(1)
print('\n✅ Replication configured!')

In [ ]:
import redis
masters = {i: redis.Redis(port=7000+i, decode_responses=True) for i in range(4)}
replicas = {i: redis.Redis(port=7004+i, decode_responses=True) for i in range(4)}

def get_slot(key):
    crc = 0
    for c in key.encode(): crc = ((crc << 5) + crc + c) & 0xFFFF
    return crc % 16384

def get_master_id(key):
    slot = get_slot(key)
    return slot // 4096

print('✅ Connected to all nodes!')

## Part 1: Relational Database

In [ ]:
import sqlite3, pandas as pd
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.executescript('''
CREATE TABLE Customer (customer_id INT PRIMARY KEY, first_name TEXT, last_name TEXT, email TEXT, phone TEXT, city TEXT, country TEXT);
CREATE TABLE Product (product_id INT PRIMARY KEY, product_name TEXT, category TEXT, price REAL, stock_quantity INT);
CREATE TABLE Order_ (order_id INT PRIMARY KEY, customer_id INT, order_date TEXT, status TEXT, total_amount REAL);
''')
cursor.executemany('INSERT INTO Customer VALUES (?,?,?,?,?,?,?)', [(1,'Ahmed','Al-Rashid','ahmed.rashid@email.com','+966501234567','Riyadh','Saudi Arabia'),(2,'Fatima','Hassan','fatima.hassan@email.com','+966502345678','Jeddah','Saudi Arabia'),(3,'Mohammed','Al-Saud','mohammed.saud@email.com','+966503456789','Dammam','Saudi Arabia'),(4,'Sara','Abdullah','sara.abdullah@email.com','+966504567890','Riyadh','Saudi Arabia'),(5,'Khalid','Omar','khalid.omar@email.com','+966505678901','Mecca','Saudi Arabia')])
cursor.executemany('INSERT INTO Product VALUES (?,?,?,?,?)', [(101,'Laptop Pro 15','Electronics',4500,25),(102,'Wireless Mouse','Electronics',150,100),(103,'Office Chair','Furniture',850,30),(104,'Standing Desk','Furniture',2200,15),(105,'Headphones','Electronics',1200,50),(106,'USB-C Hub','Electronics',280,75)])
cursor.executemany('INSERT INTO Order_ VALUES (?,?,?,?,?)', [(1001,1,'2024-06-01','delivered',4650),(1002,2,'2024-06-05','delivered',3050),(1003,1,'2024-06-10','shipped',1200),(1004,3,'2024-06-15','processing',2480),(1005,4,'2024-06-20','pending',850),(1006,5,'2024-06-25','delivered',4780),(1007,2,'2024-07-01','shipped',430)])
conn.commit()
print('✅ Database created!')

In [ ]:
print('👥 CUSTOMER TABLE')
display(pd.read_sql_query('SELECT * FROM Customer', conn))

In [ ]:
print('📦 PRODUCT TABLE')
display(pd.read_sql_query('SELECT * FROM Product', conn))

In [ ]:
print('🛒 ORDER TABLE')
display(pd.read_sql_query('SELECT * FROM Order_', conn))

## Part 2: Migration with SHARDING
**Key Format:** `TableName:TupleID:Attribute` → Value

In [ ]:
print('🚀 MIGRATING WITH SHARDING')
print('='*70)
keys_per_master = {0:[], 1:[], 2:[], 3:[]}

def set_sharded(key, value):
    mid = get_master_id(key)
    masters[mid].set(key, value)
    keys_per_master[mid].append(key)
    return mid, get_slot(key)

# Migrate Customers
print('\n👥 Migrating CUSTOMER...')
for row in cursor.execute('SELECT * FROM Customer').fetchall():
    for attr, val in [('first_name',row[1]),('last_name',row[2]),('email',row[3]),('phone',row[4]),('city',row[5]),('country',row[6])]:
        key = f'Customer:{row[0]}:{attr}'
        mid, slot = set_sharded(key, val)
        print(f'  SET {key:<35} → Master {mid} (slot {slot})')

In [ ]:
# Migrate Products
print('📦 Migrating PRODUCT...')
for row in cursor.execute('SELECT * FROM Product').fetchall():
    for attr, val in [('product_name',row[1]),('category',row[2]),('price',str(row[3])),('stock_quantity',str(row[4]))]:
        key = f'Product:{row[0]}:{attr}'
        mid, slot = set_sharded(key, val)
        print(f'  SET {key:<35} → Master {mid} (slot {slot})')

In [ ]:
# Migrate Orders
print('🛒 Migrating ORDER...')
for row in cursor.execute('SELECT * FROM Order_').fetchall():
    for attr, val in [('customer_id',str(row[1])),('order_date',row[2]),('status',row[3]),('total_amount',str(row[4]))]:
        key = f'Order:{row[0]}:{attr}'
        mid, slot = set_sharded(key, val)
        print(f'  SET {key:<35} → Master {mid} (slot {slot})')
print('\n✅ Migration complete!')

## Part 3: SHARDING DEMONSTRATION

In [ ]:
print('🎯 SHARDING - Keys Distribution')
print('='*60)
for mid in range(4):
    print(f'\nMaster {mid} (Port {7000+mid}) - {len(keys_per_master[mid])} keys')
    for k in keys_per_master[mid][:3]:
        print(f'  {k} → "{masters[mid].get(k)}"')

## Part 4: REPLICATION DEMONSTRATION
**Write to Master → Read from Replica**

In [ ]:
print('🔄 REPLICATION TEST')
print('='*60)
time.sleep(1)

# Test existing data
key = 'Customer:1:first_name'
mid = get_master_id(key)
master_val = masters[mid].get(key)
replica_val = replicas[mid].get(key)

print(f'\nKey: {key}')
print(f'Master {mid} (Port {7000+mid}): "{master_val}"')
print(f'Replica {mid} (Port {7004+mid}): "{replica_val}"')
print('✅ Data matches!' if master_val == replica_val else '❌ Mismatch')

In [ ]:
# Test NEW write
print('\n📝 Writing NEW data to Master, Reading from Replica')
print('-'*60)
new_key = 'Customer:100:first_name'
new_val = 'TestCustomer'
mid = get_master_id(new_key)

print(f'WRITE to Master {mid}: SET {new_key} "{new_val}"')
masters[mid].set(new_key, new_val)
time.sleep(1)

print(f'READ from Replica {mid}: GET {new_key}')
replica_val = replicas[mid].get(new_key)
print(f'Result: "{replica_val}"')
print('\n✅ REPLICATION VERIFIED!' if replica_val == new_val else '❌ Failed')

## Part 5: Key-Value Operations

In [ ]:
print('📋 GET by Key - Customer 1')
print('='*60)
for attr in ['first_name','last_name','email','phone','city','country']:
    key = f'Customer:1:{attr}'
    mid = get_master_id(key)
    print(f'GET {key:<30} → "{masters[mid].get(key)}"')

In [ ]:
print('📋 GET by Key - Product 101')
print('='*60)
for attr in ['product_name','category','price','stock_quantity']:
    key = f'Product:101:{attr}'
    mid = get_master_id(key)
    print(f'GET {key:<35} → "{masters[mid].get(key)}"')

## Part 6: Redis CLI Commands

In [ ]:
print('🖥️ CLI - Read from Master 7001')
!redis-cli -p 7001 GET Customer:1:first_name
!redis-cli -p 7002 GET Customer:1:email

In [ ]:
print('🖥️ CLI - Read from Replica 7005 (Same data!)')
!redis-cli -p 7005 GET Customer:1:first_name
!redis-cli -p 7006 GET Customer:1:email

In [ ]:
print('🖥️ CLI - DBSIZE on each node')
for p in range(7000,7008):
    result = !redis-cli -p {p} DBSIZE
    t = 'Master' if p < 7004 else 'Replica'
    print(f'Port {p} ({t}): {result[0]}')

## Cluster Architecture
```
MASTER 0 (7000) ←→ REPLICA 0 (7004)  Slots 0-4095
MASTER 1 (7001) ←→ REPLICA 1 (7005)  Slots 4096-8191
MASTER 2 (7002) ←→ REPLICA 2 (7006)  Slots 8192-12287
MASTER 3 (7003) ←→ REPLICA 3 (7007)  Slots 12288-16383
```

## ✅ Conclusion
| Topic | Status |
|-------|--------|
| Mapping | TableName:TupleID:Attribute → Value |
| Sharding | ✅ Keys distributed across 4 masters |
| Replication | ✅ Write Master, Read Replica works |
| Operations | SET, GET, KEYS demonstrated |